# Contents
- [Imports](#imports)
- [ANN Modelling](#model)

---
# Imports<a id=imports></a>

In [42]:
import numpy as np
import pandas as pd
import requests
import IPython.display as Disp

from sklearn.datasets import make_classification, make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score, make_scorer, roc_curve, classification_report, precision_recall_curve, roc_auc_score

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras import regularizers
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline

We import our feature engineered dataset

In [2]:
data=pd.read_csv(r'.\data\feateng.csv')

---
# ANN Modelling<a id=model></a>
Having performed our modelling and testing with traditional machine learning methods, we shall next investigate the use of neural networks for our classification problem.<br/>
From the website [machinelearningmastery.com](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/), a simple explanation of neural networks is given:
> A neural network is comprised of layers of nodes and learns to map examples of inputs to outputs.<br/>
For a given node, the inputs are multiplied by the weights in a node and summed together. This value is referred to as the summed activation of the node. The summed activation is then transformed via an activation function and defines the specific output or “activation” of the node.

As we have shown in our feature engineering, our classifier works better when it is able to capture the relations between our different features, and it seems that neural networks might be able to learn these relations on their own for better performance.<br/>
As such we shall model and test an artifical neural network for our classifier.<br/>

Let us proceed by preparing our data.

In [3]:
X=data[[x for x in data.columns if x != 'move']]
y=data['move']

We have to one-hot encode our classes for a keras multi-class classifier.

In [4]:
y = to_categorical(y)
y[:5]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

We SMOTE our data to achieve class balance, and as our ANN employs a gradient search method, we have to employ standard scaler on our datasets.

In [5]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X_res,y_res,random_state=42,stratify=y_res)

In [7]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [8]:
X_train_sc.shape

(15804, 24)

Next, we create a template for our artificial neural network. For our template, we have to decide on the number of hidden layers and our activation functions.<br/>
A good explanation/analogy of hidden layers is given in [this stats.stackexchange](https://stats.stackexchange.com/questions/63152/what-does-the-hidden-layer-in-a-neural-network-compute) post:
> Let's call the input vector x, the hidden layer activations h, and the output activation y. You have some function f that maps from x to h and another function g that maps from h to y.
<br/>So the hidden layer's activation is f(x) and the output of the network is g(f(x)).
<br/>Why have two functions (f and g) instead of just one?
<br/>If the level of complexity per function is limited, then g(f(x)) can compute things that f and g can't do individually.

However, as this project is not a mathematically rigorous investigation into predicting boxing movement, we avoid a rigorous approach to determining the number of hidden layers due to time constraints.<br/>
What we do know is that we want to capture the relationship between distances, shoulder width and spine length. As such, we set 3 hidden layers to account for any and all variations in these 3 relationships.<br/>
For our activation functions, we default to 'relu'(rectified linear unit) for our hidden layers and use 'softmax' for our output as we are performing a multiclass classification.

In [ ]:
url = 'https://www.researchgate.net/profile/Sandra_Vieira5/publication/312205163/figure/fig1/AS:453658144972800@1485171938968/a-The-building-block-of-deep-neural-networks-artificial-neuron-or-node-Each-input-x.png'
Disp.Image(request.get(url).content)
print('Example diagram of an Artificial Neural Network')

In [9]:
def model_func(input_dimen=18,layr1_neurons=18,layr1_dropout=0.5,layr2_neurons=18,layr2_dropout=0.5,layr3_neurons=18,layr3_dropout=0.5):
    model = Sequential()
    model.add(Dense(layr1_neurons,
                    activation='relu',
                    input_dim=input_dimen
                   ))
    model.add(Dropout(layr1_dropout))
    
    model.add(Dense(layr2_neurons,
                   activation='relu'
                   ))
    model.add(Dropout(layr2_dropout))
    
    model.add(Dense(layr3_neurons,
                   activation='relu'
                   ))
    model.add(Dropout(layr3_dropout))
    
    model.add(Dense(6, 
                    activation='softmax'
                   ))
    
    model.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy']
                  )
    
    return model

We then GridSearch our neural network to find the best parameters for our model.<br/>
I arbitrarily set epochs to 15 to speed up the modelling process.<br/>
For each hidden layer, I GridSearch through 24 and 30 neurons to allow for at least 24 perceptrons with our 24 features,<br/>
and for each hidden layer I GridSearch through 2 different dropout rates of 0.3 and 0.5.

In [12]:
%%time
nn = KerasClassifier(build_fn=model_func, 
                    epochs=100, 
                    verbose=1,
                    layr1_neurons=25,
                    input_dimen=X_train_sc.shape[1]
                   )

pipe = Pipeline([
    ('ss',ss),
    ('nn',nn)
])

params = {
    'nn__epochs':[15],
    'nn__layr1_neurons':[24,30],
    'nn__layr1_dropout':[0.3,0.5],
    'nn__layr2_neurons':[24,30],
    'nn__layr2_dropout':[0.3,0.5],
    'nn__layr3_neurons':[24,30],
    'nn__layr3_dropout':[0.3,0.5]
}

gs = GridSearchCV(pipe, param_grid=params)
gs.fit(X_train_sc, y_train)
print(gs.best_score_)

C:\Users\chang\.conda\envs\opencv\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/15
10536/10536 [==============================] - 15s 1ms/step - loss: 1.3880 - acc: 0.4657
Epoch 2/15
10536/10536 [==============================] - 3s 306us/step - loss: 0.8655 - acc: 0.6922
Epoch 3/15
10536/10536 [==============================] - 3s 309us/step - loss: 0.6908 - acc: 0.7648
Epoch 4/15
10536/10536 [==============================] - 3s 310us/step - loss: 0.5774 - acc: 0.8088
Epoch 5/15
10536/10536 [==============================] - 3s 311us/step - loss: 0.5190 - acc: 0.8327
Epoch 6/15
10536/10536 [==============================] - 3s 312us/step - loss: 0.4731 - acc: 0.8527
Epoch 7/15
10536/10536 [==============================] - 3s 311us/step - loss: 0.4428 - acc: 0.8615
Epoch 8/15
10536/10536 [==============================] - 3s 309us/step - loss: 0.4081 - acc: 0.8738
Epoch 9/15
10536/10536 [==============================] - 3s 306us/step - loss: 0.3946 - acc: 0.8770
Epoch 10/15
10536/10536 [==============================] - 3s 316us/step - loss: 0.3718 - ac

In [13]:
gs.best_params_

{'nn__epochs': 15,
 'nn__layr1_dropout': 0.3,
 'nn__layr1_neurons': 30,
 'nn__layr2_dropout': 0.3,
 'nn__layr2_neurons': 30,
 'nn__layr3_dropout': 0.3,
 'nn__layr3_neurons': 30}

In [14]:
gs.score(X_test,y_test)

5268/5268 [==============================] - 19s 4ms/step


0.29650721334107644

In [21]:
model = KerasClassifier(build_fn=model_func, 
                    epochs=15, 
                    verbose=1,
                    layr1_neurons=30,
                    layr2_neurons=30,
                    layr3_neurons=30,
                    layr1_dropout=0.3,
                    layr2_dropout=0.3,
                    layr3_dropout=0.3,
                    input_dimen=X_train_sc.shape[1]
                   )

In [27]:
epochs = 15 
filepath = ".\data\keras.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, verbose=1, 
     save_best_only=False)
callbacks = [checkpoint]
history=model.fit(X_train_sc, y_train, epochs=epochs, 
     shuffle=True, callbacks=callbacks)

Epoch 1/15
15804/15804 [==============================] - 53s 3ms/step - loss: 1.1499 - acc: 0.5762: 2s - loss: 1.1696 - acc

Epoch 00001: saving model to .\data\keras.hdf5
Epoch 2/15
15804/15804 [==============================] - 12s 773us/step - loss: 0.5828 - acc: 0.8088

Epoch 00002: saving model to .\data\keras.hdf5
Epoch 3/15
15804/15804 [==============================] - 12s 771us/step - loss: 0.4433 - acc: 0.8582

Epoch 00003: saving model to .\data\keras.hdf5
Epoch 4/15
15804/15804 [==============================] - 12s 757us/step - loss: 0.3757 - acc: 0.8868

Epoch 00004: saving model to .\data\keras.hdf5
Epoch 5/15
15804/15804 [==============================] - 12s 770us/step - loss: 0.3325 - acc: 0.89843s 

Epoch 00005: saving model to .\data\keras.hdf5
Epoch 6/15
15804/15804 [==============================] - 12s 772us/step - loss: 0.2954 - acc: 0.91070s - loss: 0.2957 - acc: 0.

Epoch 00006: saving model to .\data\keras.hdf5
Epoch 7/15
15804/15804 [=======================

In [32]:
history.history.keys()

dict_keys(['loss', 'acc'])

In [ ]:
train_loss_dropout = history.history['loss']
test_loss_dropout = history.history['val_loss']
plt.figure(figsize=(12, 8))
plt.plot(train_loss, label='Training loss', color='navy')
plt.plot(test_loss, label='Testing loss', color='skyblue')
plt.plot(train_loss_dropout, label='Dropout Training loss', color='darkgreen')
plt.plot(test_loss_dropout, label='Dropout Testing loss', color='lightgreen')
plt.legend();

In [ ]:
history_dropout.history['acc'][-1], history_dropout.history['val_acc'][-1]

---

In [33]:
none=pd.read_csv('../results/test/none.csv')
guard=pd.read_csv('../results/test/guard.csv')
jab=pd.read_csv('../results/test/jab.csv')
cross=pd.read_csv('../results/test/cross.csv')
hook=pd.read_csv('../results/test/hook.csv')
upcut=pd.read_csv('../results/test/uppercut.csv')

In [34]:
none['move']=0
guard['move']=1
jab['move']=2
cross['move']=3
hook['move']=4
upcut['move']=5

In [35]:
data=pd.concat([none,guard,jab,cross,hook,upcut],sort=True)
data.head()

,move,x0,x1,x10,x11,x12,x13,x14,x15,x16,...,y16,y17,y2,y3,y4,y5,y6,y7,y8,y9
0,0,0.75,0.69,NaN,0.63,0.67,NaN,0.75,NaN,0.71,...,0.12,NaN,0.26,0.54,0.79,0.27,NaN,NaN,0.68,0.90
1,0,0.75,0.68,NaN,0.62,0.63,NaN,0.75,NaN,0.71,...,0.13,NaN,0.26,0.53,0.79,0.27,NaN,NaN,0.67,0.88
2,0,0.74,0.68,NaN,0.63,0.66,NaN,0.74,NaN,0.70,...,0.13,NaN,0.26,0.52,0.79,0.27,NaN,NaN,0.67,0.89
3,0,0.74,0.67,NaN,0.62,0.64,NaN,0.74,NaN,0.70,...,0.14,NaN,0.26,0.52,0.78,0.27,NaN,NaN,0.69,0.91
4,0,0.73,0.67,NaN,0.63,NaN,NaN,0.73,NaN,0.69,...,0.14,NaN,0.27,0.52,0.78,0.27,NaN,NaN,0.69,0.91


In this run of testing, we have to create the features we created in our feature engineering section.

In [36]:
xparts=['x1','x2','x3','x4','x5','x6','x7','x8','x11']
yparts=['y1','y2','y3','y4','y5','y6','y7','y8','y11']
drop=['x0', 'x10', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x9', 'y0', 'y10', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y9']
data.drop(columns=drop,inplace=True)

data[xparts]=data[xparts].subtract(data['x1'],axis=0)
data[yparts]=data[yparts].subtract(data['y1'],axis=0)
data.fillna(0,inplace=True)

data['shwidth']=np.hypot((data.x2-data.x5),(data.y2-data.y5))
data['splength']=np.hypot((data.x1-((data.x8+data.x11)/2)),(data.y1-((data.y8+data.y11)/2)))
data['lhnsh']=np.hypot((data.x7-data.x5),(data.y7-data.y5))*data.shwidth/data.splength
data['rhnsh']=np.hypot((data.x4-data.x2),(data.y4-data.y2))*data.shwidth/data.splength
data['lelsh']=np.hypot((data.x6-data.x5),(data.y6-data.y5))*data.shwidth/data.splength
data['relsh']=np.hypot((data.x3-data.x2),(data.y3-data.y2))*data.shwidth/data.splength

In [37]:
data.head()

,move,x1,x11,x2,x3,x4,x5,x6,x7,x8,...,y5,y6,y7,y8,shwidth,splength,lhnsh,rhnsh,lelsh,relsh
0,0,0.0,-0.06,0.02,0.08,0.17,-0.03,0.0,0.0,0.02,...,0.01,0.0,0.0,0.42,0.050990,0.420476,0.003835,0.066796,0.003835,0.034726
1,0,0.0,-0.06,0.02,0.07,0.17,-0.02,0.0,0.0,0.02,...,0.01,0.0,0.0,0.41,0.041231,0.405494,0.002274,0.056008,0.002274,0.027921
2,0,0.0,-0.05,0.02,0.07,0.16,-0.02,0.0,0.0,0.00,...,0.01,0.0,0.0,0.41,0.041231,0.405771,0.002272,0.055701,0.002272,0.026903
3,0,0.0,-0.05,0.02,0.06,0.15,-0.02,0.0,0.0,0.01,...,0.00,0.0,0.0,0.42,0.041231,0.415482,0.001985,0.053191,0.001985,0.026105
4,0,0.0,-0.04,0.02,0.05,0.14,-0.02,0.0,0.0,0.01,...,0.00,0.0,0.0,0.42,0.040000,0.415271,0.001926,0.050466,0.001926,0.024253


There seem to be no issues with our dataset. Let us proceed with the testing.

In [38]:
X=data[[x for x in data.columns if x != 'move']].to_numpy()
y=data['move']

In [40]:
pred=model.predict(X)

4000/4000 [==============================] - 19s 5ms/step


In [44]:
print(classification_report(y,pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       926
           1       0.70      0.19      0.30      2271
           2       1.00      0.29      0.45       281
           3       0.00      0.00      0.00       195
           4       0.04      0.84      0.08       172
           5       0.00      0.00      0.00       155

    accuracy                           0.16      4000
   macro avg       0.29      0.22      0.14      4000
weighted avg       0.47      0.16      0.20      4000

